### Module 5B - Game of Drones: Optimization of Agricultural UAV Swarms

Written for AIFS.   
Copyright 2023 Tarek Zohdi, Emre Mengi, Omar Betancourt, Payton Goodrich. All rights reserved.

In this project, you will use machine learning / a genetic algorithm to find optimal control parameters for a swarm of autonomous vehicles tasked with mapping a set of targets quickly while avoiding collisions with obstacles and other vehicles. This problem could represent a real-world scenario like inspecting a disaster zone or construction site.

In [ ]:
################################## Importing Packages ####################################################

import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from copy import deepcopy
from matplotlib import animation
from matplotlib import rc
plt.rcParams.update({'font.size': 18})
from scipy.stats import uniform
import pandas as pd
from IPython.display import display, HTML
import Zohdi_Utils as zu


### **Problem 1:** Theory-Based Exercises ###

Answer the following questions *prior* to coding the assignment to better understand the background physics and mathematics that govern the given models. You **may** solve these problems by hand **and/or** using computational tools such as *#Python* etc. Please include all handwritten work and code used to solve each problem.

**Problem 1.1:** Analytically solve for the magnitude of the maximum possible velocity of the agents, in terms of \textit{airspeed}. \textbf{Note that airspeed is the difference between ground speed and wind speed: $\boldsymbol{v}_{AS} = (\boldsymbol{v}_i - \boldsymbol{v}_a)$.}

*Answer here*

**Problem 1.2:** Write down the Forward Euler equation for time discretization. Explain all the terms.

*Answer here*

**Problem 1.3:** In one sentence, what would happen if any of the $a$, $b$, or $c$ design variables became negative?

*Answer here*

### **Problem 2:** Coding Exercises ###  
Use the given python notebook template to complete the following coding exercises.

**Problem 2.1:** Define the constants used in the simulation. Use the variable glossary at the end of the assignment.

In [ ]:
# ################################## Problem 2.1 ####################################################

## System Parameters
Nm = #FILL IN HERE # number of initial agents
No = #FILL IN HERE # number of obstacles
Nt = #FILL IN HERE # number of targets to map

locx = 100 # x bound of target/obstacle region 
locy = 100 # y bound of target/obstacle region 
locz = 10 # z bound of target/obstacle region 

## Domain Parameters
xmax = 150 # x bound of domain
ymax = 150 # y bound of domain
zmax = 60 # z bound of domain


droneConsts = {
    
## System Parameters
'Nm' : Nm, # number of initial agents
'No' : No, # number of obstacles
'Nt' : Nt, # number of targets to map

## Physical Parameters
'Ai' : #FILL IN HERE, # agent characteristic area (m^2)
'Cdi' : #FILL IN HERE, # agent coefficient of drag
'mi' : #FILL IN HERE, # agent mass (kg)
'va' : #FILL IN HERE, # Air velocity (m/s)
'ra' : #FILL IN HERE, # Air Density (kg/m^3)
'Fp' : #FILL IN HERE, # Propolsion force magnitude (N)

## Time Stepping Parameters
'dt' : #FILL IN HERE, # time step size (s)
'tf' : #FILL IN HERE, # Maximium task time (s)

## Object Interaction Parameters
'agent_sight' : #FILL IN HERE, # maximum target mapping distance
'crash_range' : #FILL IN HERE, # agent collision distance

'w1' : #FILL IN HERE, # Weight of mapping in net cost
'w2' : #FILL IN HERE, # Weight of time usage in net cost
'w3' : #FILL IN HERE, # Weight of agent losses in net cost

## Domain Parameters
'xmax' : #FILL IN HERE, # x bound of domain
'ymax' : #FILL IN HERE, # y bound of domain
'zmax' : #FILL IN HERE, # z bound of domain

# Initial Obstacle Positions (m)
'obs' : np.hstack((uniform.rvs(-locx,2*locx,size = (No,1)),(uniform.rvs(-locy,2*locy,size = (No,1))),
                                                       (uniform.rvs(-locz,2*locz,size = (No,1))))),

# Initial Target Positions (m)
'tar' : np.hstack((uniform.rvs(-locx,2*locx,size = (Nt,1)),(uniform.rvs(-locy,2*locy,size = (Nt,1))),
                                                       (uniform.rvs(-locz,2*locz,size = (Nt,1))))),
    
# Initial Drone Positions (m)
'pos' : np.array([(xmax - 0.05*xmax)*np.ones(Nm), np.linspace(-ymax + 0.05*ymax, ymax - 0.05*ymax, Nm), 
          np.zeros(Nm)]).T,

'vel' : #FILL IN HERE, # Initial Agent velocities (m/s)

}

# ## Genetic Algorithm Parameters
K = 6 # Strings generated by breeding
P = 6 # Surviving strings for breeding
S = 20 # Design strings per generation
G = 100 # Total Generations
SB = np.hstack((np.zeros((15,1)),2*np.ones((15,1)))) # Same search bounds for all design strings

# ##################################################################################################################

In [ ]:
################################## Plotting Initial System ####################################################

zu.PlotInitialDroneSystem(droneConsts)

**Problem 2.2:** Run the genetic algorithm.

A good set of control parameters will enable agents to map targets quickly and completely without crashes. The cost function for comparing the outputs from different parameters is: 

\begin{equation}
    \Pi = w_1M^* + w_2T^* + w_3L^* 
\end{equation}

\begin{equation}
    M^* = \frac{\text{(Unmapped targets)}}{\text{(Total targets)}}, \;\; T^* = \frac{\text{(Used time)}}{\text{(Total time)}}, \;\; L^* = \frac{\text{(Crashed agents)}}{\text{(Total agents)}}
\end{equation}

\begin{equation*}
    w_1 = 70, \;\; w_2 = 10, \;\; w_3 = 20
\end{equation*}

Note that all terms in the cost function are non-dimensional. The weights reflect the relative importance of each term. The ``design string" for this problem contains all 15 undetermined constants: 

\begin{equation} 
\Lambda^{i}= \{\Lambda^i_1,..., \Lambda^i_N\}= \{ W_{mt}, W_{mo}, W_{mm}, w_{t1}, w_{t2}, w_{o1}, w_{o2}, w_{m1}, w_{m2}, a_1, a_2,  b_1, b_2,  c_1, c_2\}^i
\end{equation} 

You should initially assume that the values of all design parameters lie in the interval: 

\begin{equation}
    0 \leq \Lambda_n \leq 2 \;\forall \; n
\end{equation}

Scale and offset all randomly-generated design strings to span this interval.

In [ ]:
################################# Problem 2.2 #######################################################
def myGA(S,G,P,SB,Func,consts):
    
    # Get number of design variables from size of search bound array
    dv = SB.shape[0] 
    
    # set number of kids (K) equal to number of parents
    K = P
    
    # Initialize all variables to be saved
    Min = np.zeros(G) # Minimum cost for each generation
    PAve = np.zeros(G) # Parent average for each generation
    Ave = np.zeros(G) # Total population average for each generation
    lamHist = np.zeros((G,dv))    
    
    
    Pi = np.zeros(S) # All costs in an individual generation
    
    # Initialize Lam array which will contain all of the design strings for all design variables
    Lam = np.zeros((S,dv))
    
    # Generate initial random population by looping through the number of design variables and building out the Lam array
    for i in range(dv):
        Lam[:,i] = uniform.rvs(SB[i,0],SB[i,1]-SB[i,0],size = S)
        
    # In first generation, calculate cost for all strings.
    # After, only calculate new strings since fitness for top P parents are already calculated
    # Initialize an index parameter to 0 to track which design string to start evaluating costs from
    start = 0 
    
    for i in range(G): # Loop through generations
        
        # Calculate fitness of unknown design string costs
        for j in range(start,S): # Evaluate fitness of strings
            
            # Plug in design string control variables and array of function constants
            output = Func(Lam[j,:], consts) # Outputs dict of function outputs
            
            Pi[j] = output['Pi'] # Extract cost from dict of outputs and assign it to cost array
            
        
        # Sort cost and design strings based on performance
        ind = np.argsort(Pi)
        Pi = np.sort(Pi)
        Lam = Lam[ind,:]
        
        # Save best design string for current generation
        lamHist[i,:] = Lam[0,:]
        
        # Generate offspring radnom parameters and indices for vectorized offspring calculation
        phi = np.random.rand(K,SB.shape[0]) # Generate random weights for offspring
        ind1 = range(0,K,2) # First set of children based on even numbered parameters
        ind2 = range(1,K,2) # Second set of children based on odd numbered parameters
        
        Parents = Lam[0:P,:] # Top P performing parents
        Children1 = phi[ind1,:]*Lam[ind1,:] + (1-phi[ind1,:])*Lam[ind2,:] # First set of children
        Children2 = phi[ind2,:]*Lam[ind2,:] + (1-phi[ind2,:])*Lam[ind1,:] # Second set of children
        
        # Initialize newPopulation array which will have S-P-K new random strings for all design variables
        newPop = np.zeros((S-P-K,dv))
        
        # Generate S - P - K new random strings by looping through the number of design variables and building out the new Population array
        for j in range(dv):
            newPop[:,j] = uniform.rvs(SB[j,0],SB[j,1]-SB[j,0],size = S-P-K)
        
         # Vertically stack parents, children, and new strings to use in next generation  
        Lam = np.vstack((Parents, Children1, Children2, newPop))  
        
        # Save minimum, parent average, and population average cost values for plotting
        Min[i] = Pi[0]
        PAve[i] = np.mean(Pi[0:P])
        Ave[i] = np.mean(Pi)
        
        # Update index parameter to P such that only new string cost values are calculated
        start = P
        
        # Print miminum value of cost for debugging (should monotonically decrease over generations)
        print("Best cost for generation " + str(i+1) + ": " + str(Min[i]))
        
    bestLam = Lam[0,:] # Extract best design string parameters afer all generations are run
    
    return(lamHist, Lam, bestLam, Pi, Min, PAve, Ave)

In [ ]:
################################# Call GA to optimize drone system #######################################################

lamHist, Lam, bestLam, Pi, Min, PAve, Ave = myGA(S,G,P,SB,zu.droneSim,droneConsts)

### **Problem 3:** Analyzing Your Results ###  
Answer the following questions about the code you created.

**Problem 3.1:** Report your best-performing 4 designs in a table similar to the following, but replacing $\Lambda_i$ with the design variables specific to this project. Use *pandas DataFrame* to generate the table in cell *Problem 3.1*  

<img src="GAtable.png" width="400" />

In [ ]:
################################# Problem 3.1 #######################################################

# Create the requested table using pandas data frame
GAoutputTable = {
    'Design': ['1', '2','3','4'],
    #FILL IN THE REST OF THE COLUMNS HERE
}

df = pd.DataFrame(GAoutputTable)

display(HTML(df.to_html(index=False)))

**Problem 3.2:** Provide a convergence plot showing the total cost of the best design, the mean of all parent designs, and the mean of the overall population for each generation. Provide a plot showing the individual performance components (i.e., plot $M^*$, $T^*$, and $L^*$), for the overall best performer. Do so by running cell *Problem 3.2/3.3*. Discuss any important observations.

In [ ]:
################################# Problem 3.2/3.3 #######################################################

zu.PlotDroneGAStatistics(G,Min,PAve,Ave,lamHist,droneConsts)

You can also visualize the best drone behavior by running the animation below.

In [ ]:
################################# Plotting Best Solution Animation ###############################################################

zu.PlotDroneAnimation(bestLam,droneConsts)

Use the following variable glossary to fill in the required variables.

<img src="VariableGlossary.png" width="800" />